In [ ]:
import silence_tensorflow.auto
import tensorflow as tf
gpuDevices = tf.config.list_physical_devices('GPU')
print("Num GPUs Available: ", len(gpuDevices))
tf.config.experimental.set_memory_growth(gpuDevices[0], True)
tf.config.optimizer.set_experimental_options({'layout_optimizer': False})
import keras
import os, sys, random, glob, time, logging, io
from PIL import Image

import transformers
from IPython import display
import datasets
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
ts = lambda tensor : keras.preprocessing.image.array_to_img(tensor)

def plot_n_samples(dataset):
    plt.figure(figsize=(20, 4))
    for i in range(len(dataset)):
        # assert tf.is_tensor(dataset[i]), "This ain't a tensor"
        ax = plt.subplot(2, len(dataset), i + 1)
        plt.imshow(ts(dataset[i].numpy()))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.show()

In [ ]:
def convert_image_gray(dp) :
    image = dp['image'].convert("L")
    buffer = io.BytesIO()
    image.save(buffer, format="jpeg", compress_level="1")
    return {"image": {"bytes" : buffer.getvalue()}}

# to_grayscale_pillow_np = lambda x:{'gray_img' : np.expand_dims(np.asarray(x['image'].convert("L"), dtype=np.uint8), axis=-1)}
# map_one_hot= lambda x : {'solution_onehot' : {
#         'digit1': int(x['solution'][0]),
#         'digit2': int(x['solution'][1]),
#         'digit3': int(x['solution'][2]),
#         'digit4': int(x['solution'][3]),
#         'digit5': int(x['solution'][4]),
#         'digit6': int(x['solution'][5]),
#     }}
ds = datasets.load_dataset("project-sloth/captcha-images")
ds = ds.map(convert_image_gray, writer_batch_size=50)
# ds = ds.remove_columns("image")
# ds = ds.map(lambda x:{'solution': [keras.utils.to_categorical(digit, num_classes=10) for digit in x['solution']]}, keep_in_memory=True)
# ds = ds.map(map_one_hot)
# ds = ds.remove_columns("solution")
ds = ds.with_format("tensorflow")

In [ ]:
print(ds['train'].)

In [ ]:
ds4 = ds.map(lambda x: {'asdfjlhsadf' : tf.reshape(x['image'], [50, 200, 1])})

In [ ]:
ds['train'].features

In [ ]:
mtp = lambda mat: print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in mat]))

In [ ]:
print(np.transpose(ds['train'][0]['solution']))

In [ ]:
dstf_train = ds['train'].to_tf_dataset(
            columns=["gray_img"],
            label_cols=["solution"],
            batch_size=64
            )

In [ ]:
# print(ds['test'][0]['image'].device)
# print(np.array(ds['test'][0]['gray_img']).shape)
# print(np.array(ds['test'][0]['solution']).shape)
# plot_n_samples(ds['train'][:4]['gray_img'])
# plot_n_samples(ds['train'][:4]['image'])

# data_in_memory = ds['train'][:]

In [ ]:
def make_encoder() :
    model = keras.Sequential()
    input = keras.layers.Input(shape=(50,200,1))
    model.add(input)
    model.add(keras.layers.Conv2D(128,(3,3), activation='relu', padding='same'))
    assert model.input_shape == (None, 50,200,1)
    
    model.add(keras.layers.MaxPool2D((2,2)))
    model.add(keras.layers.Flatten())

    # print(model.output_shape)

    model.add(keras.layers.Dense(128,activation='relu'))
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Dense(64, activation='relu'))

    # print(model.output_shape)    

    # outputs = []
    # for i in range(6):
    #     outputs.append(keras.layers.Dense(10, activation='softmax', name=f'digit{i+1}')(model.layers[-1].output))
    model.add(keras.layers.Dense(60, activation='softmax'))
    output = keras.layers.Reshape((6,10))(model.layers[-1].output)
    
    functional_model = keras.Model(inputs=input, outputs=output)
    
    # print (functional_model.output)
    return functional_model
    

In [ ]:
model = make_encoder()

In [ ]:
print(model.output_shape)

In [1]:
model.summary()

NameError: name 'model' is not defined

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
import tensorboard

In [2]:
model.fit(dstf_train, epochs=5, batch_size=10)

NameError: name 'model' is not defined